In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_ChanRed, CapsEEGNet
import torch
import os

In [2]:
model = EEGNet(nb_classes = 2, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  50946


In [3]:
model_SEED = EEGNet_ChanRed(nb_classes = 3, Chans = 62, InnerChans=14, Samples = 200, dropoutRate = 0.1, kernLength = 25, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model_SEED.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  53045


In [4]:
model_capseegnet = CapsEEGNet(nb_classes=2)

total_params = sum(p.numel() for p in model_capseegnet.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  529441


In [5]:
x = torch.randn(8, 1, 14, 128)
out = model_capseegnet(x)
print(out.shape)
print(out)

torch.Size([8, 2])
tensor([[0.5002, 0.4998],
        [0.5001, 0.4999],
        [0.5002, 0.4998],
        [0.5001, 0.4999],
        [0.5001, 0.4999],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5001, 0.4999]], grad_fn=<SoftmaxBackward0>)


/users/eleves-a/2021/julien.gadonneix/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  return F.conv2d(input, weight, bias, self.stride,


In [6]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

16
1
cuda
True
_CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24218MB, multi_processor_count=64)


In [7]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


FileNotFoundError: [Errno 2] No such file or directory: 'system_profiler'

In [ ]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.7246112823486328


In [ ]:
t = torch.randn(8,8)
print(t)
torch.reshape(t, (-1, 2, 8))

tensor([[-0.4993,  1.0841,  0.0559,  0.1679, -0.5550, -0.0980, -0.9082, -0.6369],
        [ 0.5351,  1.4419,  0.1808,  1.9187,  1.6569, -0.5565,  0.3886, -0.2302],
        [ 2.0292,  0.8219, -1.5713,  0.0808,  0.5523, -0.5211, -1.1286, -0.1915],
        [ 0.2730,  2.3373,  0.3642,  0.5787, -1.8168, -1.2895, -1.5716, -1.8972],
        [ 0.7142, -0.1486,  1.1180,  0.2327,  1.3839, -0.9730,  0.8136, -0.0598],
        [ 1.4267,  0.5381,  1.3016, -1.2277,  0.0903,  0.4886,  0.5824, -0.6297],
        [ 0.9264,  0.7906,  1.5842, -0.5572,  0.0211,  0.4384, -0.3296,  0.4292],
        [-0.9650,  0.1832,  0.6145,  0.3053,  0.1119,  0.7097, -0.3226,  0.0037]])


tensor([[[-0.4993,  1.0841,  0.0559,  0.1679, -0.5550, -0.0980, -0.9082,
          -0.6369],
         [ 0.5351,  1.4419,  0.1808,  1.9187,  1.6569, -0.5565,  0.3886,
          -0.2302]],

        [[ 2.0292,  0.8219, -1.5713,  0.0808,  0.5523, -0.5211, -1.1286,
          -0.1915],
         [ 0.2730,  2.3373,  0.3642,  0.5787, -1.8168, -1.2895, -1.5716,
          -1.8972]],

        [[ 0.7142, -0.1486,  1.1180,  0.2327,  1.3839, -0.9730,  0.8136,
          -0.0598],
         [ 1.4267,  0.5381,  1.3016, -1.2277,  0.0903,  0.4886,  0.5824,
          -0.6297]],

        [[ 0.9264,  0.7906,  1.5842, -0.5572,  0.0211,  0.4384, -0.3296,
           0.4292],
         [-0.9650,  0.1832,  0.6145,  0.3053,  0.1119,  0.7097, -0.3226,
           0.0037]]])